샘플용 파일 생성다항식

g1(x) = X^3 + X + 1

g2(x) = X^2 + 1

In [69]:
with open('decoded_by_hardencoded_helloworld_test2.bin' ,'rb') as f:
    data = f.read()

In [70]:
plain_text = str()

for i in range(len(data)):
    plain_text += chr(data[i])

In [71]:
plain_text

'\x8f?F\x1b!\x03Ô!~F\x1f· '

In [72]:
data

b'\x8f?F\x1b!\x03\xd4!~F\x1f\xb7 '

In [88]:
def _oct2bin(octal_poly, K):
    """MATLAB 스타일: 옥탈 다항식을 K 길이의 이진 리스트(MSB-first)로 변환합니다."""
    # bin(int(string, base))[2:]를 통해 '0o'와 '0b' 접두사를 제거하고 이진 문자열을 얻습니다.
    bits_str = bin(int(str(octal_poly), 8))[2:]

    if len(bits_str) > K:
        raise ValueError(f"옥탈 {octal_poly}의 길이가 K={K}를 초과합니다.")

    padded_bits = [0] * (K - len(bits_str)) + [int(b) for b in bits_str]
    return padded_bits

In [91]:
_oct2bin('0o11', 4)

[1, 0, 0, 1]

In [96]:
type(_oct2bin('0o5', 4))

list

In [99]:
bin(0o171)

'0b1111001'

In [104]:
a = '1,1,1,0,0,1'

In [114]:
b = list(map(int, a.split(',')))

In [115]:
b

[1, 1, 1, 0, 0, 1]